<a href="https://colab.research.google.com/github/maorgreshler/NN_google_colab/blob/main/certified_NN_wo_plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import io
from google.colab import files
uploaded = files.upload()

df = pd.read_csv(io.BytesIO(uploaded['vehicle_simulation_data_random_test.csv']))
print(df.head())

In [2]:
input_columns = ["velocity", "yaw_angle", "yaw_rate", "slip_angle", "front_wheel", "back_wheel" , "acceleration", "angular_acceleration"]

output_columns = ["delta_x_position", "delta_y_position", "delta_theta", "delta_velocity", "delta_yaw_angle",
                  "delta_yaw_rate", "delta_slip_angle", "delta_front_wheel", "delta_back_wheel"]

input_data = df[input_columns]
output_data = df[output_columns]

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Convert pandas DataFrames to numpy arrays
X = input_data.values
y = output_data.values
# Normalize the data
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instantiate the model, define loss function and optimizer
input_size = X_train.shape[1]
output_size = y_train.shape[1]
model = MLP(input_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 0.9150
Epoch [20/100], Loss: 0.8142
Epoch [30/100], Loss: 0.7178
Epoch [40/100], Loss: 0.6608
Epoch [50/100], Loss: 0.6349
Epoch [60/100], Loss: 0.6113
Epoch [70/100], Loss: 0.5879
Epoch [80/100], Loss: 0.5658
Epoch [90/100], Loss: 0.5469
Epoch [100/100], Loss: 0.5323


In [10]:
# Evaluate the model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

with torch.no_grad():
    predictions = model(X_test_tensor)
    predictions = scaler_y.inverse_transform(predictions.numpy())

print("Predictions:")
print(predictions)

Test Loss: 0.4983
Predictions:
[[ 7.30388761e-02 -6.64709210e-02 -2.80898414e-03 ...  1.19044434e-03
  -7.64075756e+00 -9.25350761e+00]
 [ 1.05628952e-01 -1.12319835e-01  2.78299302e-03 ...  3.14143440e-03
  -3.92455840e+00 -5.25352716e+00]
 [-3.57321277e-02  3.68379056e-02 -4.11404035e-04 ... -7.18784472e-03
  -2.95370412e+00 -3.88876605e+00]
 ...
 [-5.86761795e-02  8.96647666e-03  2.99529085e-04 ... -9.70532477e-04
  -6.80837488e+00 -8.28355312e+00]
 [-4.49123457e-02 -5.27048996e-03 -2.61643459e-03 ...  3.83947999e-03
  -1.39751685e+00 -3.78355837e+00]
 [ 5.90105206e-02  7.88538381e-02  7.12402165e-04 ... -7.51594175e-03
  -2.91897964e+00 -3.40619349e+00]]
